In [2]:
import os
import cv2
import numpy as np
import argparse
import warnings
import time

from src.anti_spoof_predict import AntiSpoofPredict
from src.generate_patches import CropImage
from src.utility import parse_model_name
warnings.filterwarnings('ignore')


In [29]:
path = r"./images/Facial Liveliness/"

In [2]:
# from glob import glob # glob use to extract path of file

In [3]:
# path = glob(r'./data/images/*.jpg')

In [37]:
img111 = cv2.imread(path[1])

In [40]:

def check_image(img):
    height, width, channel = img.shape
    if width/height != 3/4:
        print("Image is not appropriate!!!\nHeight/Width should be 4/3.")
        return False
    else:
        return True



def liveliness_test(image_name, model_dir, device_id):
        
    model_test = AntiSpoofPredict(device_id)
    image_cropper = CropImage()
    img = cv2.imread(path + image_name)
    result = check_image(img)
    if result is False:
        return
    image_bbox = model_test.get_bbox(img)
    prediction = np.zeros((1, 3))
    test_speed = 0
    # sum the prediction from single model's result
    for model_name in os.listdir(model_dir):
        h_input, w_input, model_type, scale = parse_model_name(model_name)
        param = {
            "org_img": img,
            "bbox": image_bbox,
            "scale": scale,
            "out_w": w_input,
            "out_h": h_input,
            "crop": True,
        }
        if scale is None:
            param["crop"] = False
        img1 = image_cropper.crop(**param)
        start = time.time()
        prediction += model_test.predict(img1, os.path.join(model_dir, model_name))
        test_speed += time.time()-start

    # draw result of prediction
    label = np.argmax(prediction)
    value = prediction[0][label]/2
    if label == 1:
        print("User '{}' is Verified. Score: {:.2f}.".format(image_name, value))
        result_text = "RealFace Score: {:.2f}".format(value)
        color = (255, 0, 0)
        print("verification completed in {:.2f} s".format(test_speed))
    else:
        print("Dear User, it looks like you are using a fake image. Please start the verification process again.")
        print("Score: {:.2f}".format(value))
        result_text = "FakeFace Score: {:.2f}".format(value)
        color = (0, 0, 255)
        print("Operation completed in {:.2f} seconds".format(test_speed))
    cv2.rectangle(
        img,
        (image_bbox[0], image_bbox[1]),
        (image_bbox[0] + image_bbox[2], image_bbox[1] + image_bbox[3]),
        color, 2)
    cv2.putText(
        img,
        result_text,
        (image_bbox[0], image_bbox[1] - 5),
        cv2.FONT_HERSHEY_COMPLEX, 0.5*img.shape[0]/1024, color)

    format_ = os.path.splitext(image_name)[-1]
    result_image_name = image_name.replace(format_, "_result" + format_)
    cv2.imwrite(path + result_image_name, img)



if __name__ == "__main__":
    desc = "test"
    parser = argparse.ArgumentParser(description=desc)
    parser.add_argument(
        "--device_id",
        type=int,
        default=0,
        help="which gpu id, [0/1/2/3]")
    parser.add_argument(
        "--model_dir",
        type=str,
        default="./resources/anti_spoof_models",
        help="model_lib used to test")
    parser.add_argument(
        "--image_name",
        type=str,
        default="image_F1.jpg",
        help="image used to test")
    
    # Use parse_known_args instead of parse_args
    args, unknown = parser.parse_known_args()
    
    #test(args.image_name, args.model_dir, args.device_id)


In [41]:
liveliness_test(args.image_name, args.model_dir, args.device_id)

Dear User, it looks like you are using a fake image. Please start the verification process again.
Score: 0.73
Operation completed in 0.25 seconds
